<a href="https://colab.research.google.com/github/judemoon/dl_tensorflow/blob/master/Copy_of_Classifying_Images_of_Clothing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Classifying Images of Clothing

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_for_deep_learning/l03c01_classifying_images_of_clothing.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_for_deep_learning/l03c01_classifying_images_of_clothing.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

In this tutorial, we'll build and train a neural network to classify images of clothing, like sneakers and shirts.

It's okay if you don't understand everything. This is a fast-paced overview of a complete TensorFlow program, with explanations along the way. The goal is to get the general sense of a TensorFlow project, not to catch every detail.

This guide uses [tf.keras](https://www.tensorflow.org/guide/keras), a high-level API to build and train models in TensorFlow.

## Install and import dependencies

We'll need [TensorFlow Datasets](https://www.tensorflow.org/datasets/), an API that simplifies downloading and accessing datasets, and provides several sample datasets to work with. We're also using a few helper libraries.

In [0]:
!pip install -U tensorflow_datasets

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [0]:
try:
  # Use the %tensorflow_version magic if in colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

In [0]:
# Import TensorFlow Datasets
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

# Helper libraries
import math
import numpy as np
import matplotlib.pyplot as plt

In [0]:
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

## Import the Fashion MNIST dataset

This guide uses the [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist) dataset, which contains 70,000 grayscale images in 10 categories. The images show individual articles of clothing at low resolution (28 $\times$ 28 pixels), as seen here:

<table>
  <tr><td>
    <img src="https://tensorflow.org/images/fashion-mnist-sprite.png"
         alt="Fashion MNIST sprite" width="600">
  </td></tr>
  <tr><td align="center">
    <b>Figure 1.</b> <a href="https://github.com/zalandoresearch/fashion-mnist">Fashion-MNIST samples</a> (by Zalando, MIT License).<br/>&nbsp;
  </td></tr>
</table>

Fashion MNIST is intended as a drop-in replacement for the classic [MNIST](http://yann.lecun.com/exdb/mnist/) dataset—often used as the "Hello, World" of machine learning programs for computer vision. The MNIST dataset contains images of handwritten digits (0, 1, 2, etc) in an identical format to the articles of clothing we'll use here.

This guide uses Fashion MNIST for variety, and because it's a slightly more challenging problem than regular MNIST. Both datasets are relatively small and are used to verify that an algorithm works as expected. They're good starting points to test and debug code.

We will use 60,000 images to train the network and 10,000 images to evaluate how accurately the network learned to classify images. You can access the Fashion MNIST directly from TensorFlow, using the [Datasets](https://www.tensorflow.org/datasets) API:

In [0]:
dataset, metadata = tfds.load('fashion_mnist', as_supervised=True, with_info=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

In [7]:
metadata

tfds.core.DatasetInfo(
    name='fashion_mnist',
    version=3.0.0,
    description='Fashion-MNIST is a dataset of Zalando's article images consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes.',
    homepage='https://github.com/zalandoresearch/fashion-mnist',
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=70000,
    splits={
        'test': 10000,
        'train': 60000,
    },
    supervised_keys=('image', 'label'),
    citation="""@article{DBLP:journals/corr/abs-1708-07747,
      author    = {Han Xiao and
                   Kashif Rasul and
                   Roland Vollgraf},
      title     = {Fashion-MNIST: a Novel Image Dataset for Benchmarking Machine Learning
                   Algorithms},
      journal   = {CoRR},
      volume

In [8]:
dataset

{'test': <DatasetV1Adapter shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>,
 'train': <DatasetV1Adapter shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>}

In [9]:
dataset['test']

<DatasetV1Adapter shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>

Loading the dataset returns metadata as well as a *training dataset* and *test dataset*.

* The model is trained using `train_dataset`.
* The model is tested against `test_dataset`.

The images are 28 $\times$ 28 arrays, with pixel values in the range `[0, 255]`. The *labels* are an array of integers, in the range `[0, 9]`. These correspond to the *class* of clothing the image represents:

<table>
  <tr>
    <th>Label</th>
    <th>Class</th>
  </tr>
  <tr>
    <td>0</td>
    <td>T-shirt/top</td>
  </tr>
  <tr>
    <td>1</td>
    <td>Trouser</td>
  </tr>
    <tr>
    <td>2</td>
    <td>Pullover</td>
  </tr>
    <tr>
    <td>3</td>
    <td>Dress</td>
  </tr>
    <tr>
    <td>4</td>
    <td>Coat</td>
  </tr>
    <tr>
    <td>5</td>
    <td>Sandal</td>
  </tr>
    <tr>
    <td>6</td>
    <td>Shirt</td>
  </tr>
    <tr>
    <td>7</td>
    <td>Sneaker</td>
  </tr>
    <tr>
    <td>8</td>
    <td>Bag</td>
  </tr>
    <tr>
    <td>9</td>
    <td>Ankle boot</td>
  </tr>
</table>

Each image is mapped to a single label. Since the *class names* are not included with the dataset, store them here to use later when plotting the images:

In [0]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal',      'Shirt',   'Sneaker',  'Bag',   'Ankle boot']

### Explore the data

Let's explore the format of the dataset before training the model. The following shows there are 60,000 images in the training set, and 10000 images in the test set:

In [0]:
num_train_examples = metadata.splits['train'].num_examples
num_test_examples = metadata.splits['test'].num_examples
print("Number of training examples: {}".format(num_train_examples))
print("Number of test examples:     {}".format(num_test_examples))

## Preprocess the data

The value of each pixel in the image data is an integer in the range `[0,255]`. For the model to work properly, these values need to be normalized to the range `[0,1]`. So here we create a normalization function, and then apply it to each image in the test and train datasets.

In [0]:
def normalize(images, labels):
  images = tf.cast(images, tf.float32)
  images /= 255
  return images, labels

# The map function applies the normalize function to each element in the train
# and test datasets
norm_train_dataset =  train_dataset.map(normalize)
norm_test_dataset  =  test_dataset.map(normalize)

# The first time you use the dataset, the images will be loaded from disk
# Caching will keep them in memory, making training faster
train_dataset =  train_dataset.cache()
test_dataset  =  test_dataset.cache()

norm_train_dataset = norm_train_dataset.cache()
norm_test_dataset = norm_test_dataset.cache()


### Explore the processed data

Let's plot an image to see what it looks like.

In [0]:
for image, label in test_dataset.take(1):
  print(image)
  #print(label)

In [21]:
image.shape

TensorShape([28, 28, 1])

In [22]:
image.numpy().reshape((28,28)).shape

(28, 28)

In [0]:
# Take a single image, and remove the color dimension by reshaping
for image, label in test_dataset.take(1):
  break
image = image.numpy().reshape((28,28))

# Plot the image - voila a piece of fashion clothing
plt.figure()
plt.imshow(image, cmap=plt.cm.binary)
plt.colorbar()
plt.grid(False)
plt.show()

Display the first 25 images from the *training set* and display the class name below each image. Verify that the data is in the correct format and we're ready to build and train the network.

In [0]:
plt.figure(figsize=(10,10))
i = 0
for (image, label) in test_dataset.take(25):
    image = image.numpy().reshape((28,28))
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(image, cmap=plt.cm.binary)
    plt.xlabel(class_names[label])
    i += 1
plt.show()

## Build the model

Building the neural network requires configuring the layers of the model, then compiling the model.

### Setup the layers

The basic building block of a neural network is the *layer*. A layer extracts a representation from the data fed into it. Hopefully, a series of connected layers results in a representation that is meaningful for the problem at hand.

Much of deep learning consists of chaining together simple layers. Most layers, like `tf.keras.layers.Dense`, have internal parameters which are adjusted ("learned") during training.

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

This network has three layers:

* **input** `tf.keras.layers.Flatten` — This layer transforms the images from a 2d-array of 28 $\times$ 28 pixels, to a 1d-array of 784 pixels (28\*28). Think of this layer as unstacking rows of pixels in the image and lining them up. This layer has no parameters to learn, as it only reformats the data.

* **"hidden"** `tf.keras.layers.Dense`— A densely connected layer of 128 neurons. Each neuron (or node) takes input from all 784 nodes in the previous layer, weighting that input according to hidden parameters which will be learned during training, and outputs a single value to the next layer.

* **output** `tf.keras.layers.Dense` — A 10-node *softmax* layer, with each node representing a class of clothing. As in the previous layer, each node takes input from the 128 nodes in the layer before it. Each node weights the input according to learned parameters, and then outputs a value in the range `[0, 1]`, representing the probability that the image belongs to that class. The sum of all 10 node values is 1.


### Compile the model

Before the model is ready for training, it needs a few more settings. These are added during the model's *compile* step:


* *Loss function* — An algorithm for measuring how far the model's outputs are from the desired output. The goal of training is this measures loss.
* *Optimizer* —An algorithm for adjusting the inner parameters of the model in order to minimize loss.
* *Metrics* —Used to monitor the training and testing steps. The following example uses *accuracy*, the fraction of the images that are correctly classified.

In [0]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

## Train the model

First, we define the iteration behavior for the train dataset:
1. Repeat forever by specifying `dataset.repeat()` (the `epochs` parameter described below limits how long we perform training).
2. The `dataset.shuffle(60000)` randomizes the order so our model cannot learn anything from the order of the examples.
3. And `dataset.batch(32)` tells `model.fit` to use batches of 32 images and labels when updating the model variables.

Training is performed by calling the `model.fit` method:
1. Feed the training data to the model using `train_dataset`.
2. The model learns to associate images and labels.
3. The `epochs=5` parameter limits training to 5 full iterations of the training dataset, so a total of 5 * 60000 = 300000 examples.

(Don't worry about `steps_per_epoch`, the requirement to have this flag will soon be removed.)

In [0]:
# Warning: run this code block only one time, otherwise, input_shape error will occur during the fitting
BATCH_SIZE = 32

train_dataset = train_dataset.repeat().shuffle(num_train_examples).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

norm_train_dataset = norm_train_dataset.repeat().shuffle(num_train_examples).batch(BATCH_SIZE)
norm_test_dataset = norm_test_dataset.batch(BATCH_SIZE)

In [0]:
model.fit(norm_train_dataset, epochs=5, steps_per_epoch=math.ceil(num_train_examples/BATCH_SIZE))

As the model trains, the loss and accuracy metrics are displayed. This model reaches an accuracy of about 0.88 (or 88%) on the training data.

## Evaluate accuracy

Next, compare how the model performs on the test dataset. Use all examples we have in the test dataset to assess accuracy.

In [0]:
test_loss, test_accuracy = model.evaluate(norm_test_dataset, steps=math.ceil(num_test_examples/32))
print('Accuracy on test dataset:', test_accuracy)

As it turns out, the accuracy on the test dataset is smaller than the accuracy on the training dataset. This is completely normal, since the model was trained on the `train_dataset`. When the model sees images it has never seen during training, (that is, from the `test_dataset`), we can expect performance to go down. 

## Make predictions and explore

With the model trained, we can use it to make predictions about some images.

In [0]:
for test_images, test_labels in test_dataset.take(1):
  test_images = test_images.numpy()
  test_labels = test_labels.numpy()
  predictions = model.predict(test_images)

In [0]:
predictions.shape


Here, the model has predicted the label for each image in the testing set. Let's take a look at the first prediction:

In [0]:
predictions[0]

A prediction is an array of 10 numbers. These describe the "confidence" of the model that the image corresponds to each of the 10 different articles of clothing. We can see which label has the highest confidence value:

In [0]:
np.argmax(predictions[0])

So the model is most confident that this image is a shirt, or `class_names[6]`. And we can check the test label to see this is correct:

In [0]:
test_labels[0]

We can graph this to look at the full set of 10 class predictions

In [0]:
def plot_image(i, predictions_array, true_labels, images):
  predictions_array, true_label, img = predictions_array[i], true_labels[i], images[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  
  plt.imshow(img[...,0], cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'
  
  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array[i], true_label[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1]) 
  predicted_label = np.argmax(predictions_array)
  
  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

Let's look at the 0th image, predictions, and prediction array. 

In [0]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions, test_labels)

In [0]:
i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions, test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions, test_labels)

Let's plot several images with their predictions. Correct prediction labels are blue and incorrect prediction labels are red. The number gives the percent (out of 100) for the predicted label. Note that it can be wrong even when very confident. 

In [0]:
# Plot the first X test images, their predicted label, and the true label
# Color correct predictions in blue, incorrect predictions in red
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions, test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions, test_labels)


Finally, use the trained model to make a prediction about a single image. 

In [0]:
# Grab an image from the test dataset
img = test_images[0]

print(img.shape)

`tf.keras` models are optimized to make predictions on a *batch*, or collection, of examples at once. So even though we're using a single image, we need to add it to a list:

In [0]:
# Add the image to a batch where it's the only member.
img = np.array([img])

print(img.shape)

Now predict the image:

In [0]:
predictions_single = model.predict(img)

print(predictions_single)

In [0]:
plot_value_array(0, predictions_single, test_labels)
_ = plt.xticks(range(10), class_names, rotation=45)

`model.predict` returns a list of lists, one for each image in the batch of data. Grab the predictions for our (only) image in the batch:

In [0]:
np.argmax(predictions_single[0])

And, as before, the model predicts a label of 6 (shirt).

# Exercises

Experiment with different models and see how the accuracy results differ. In particular change the following parameters:
*   Set training epochs set to 1
*   Number of neurons in the Dense layer following the Flatten one. For example, go really low (e.g. 10) in ranges up to 512 and see how accuracy changes
*   Add additional Dense layers between the Flatten and the final `Dense(10)`, experiment with different units in these layers
*   Don't normalize the pixel values, and see the effect that has


Remember to enable GPU to make everything run faster (Runtime -> Change runtime type -> Hardware accelerator -> GPU).
Also, if you run into trouble, simply reset the entire environment and start from the beginning:
*   Edit -> Clear all outputs
*   Runtime -> Reset all runtimes

In [0]:
def build_model(hidden1_num=128, hidden2_num=128, hidden2=False):

  # Setup the layers
  if hidden2:
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
        tf.keras.layers.Dense(hidden1_num, activation=tf.nn.relu),
        tf.keras.layers.Dense(hidden2_num, activation=tf.nn.relu),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])

  else:
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
        tf.keras.layers.Dense(hidden1_num, activation=tf.nn.relu),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])

  # Compile the model
  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

## Set training epochs set to 1

In [11]:
model_v2 = build_model()

# Train the model
# The epochs=1 parameter limits training to 1 full iteration of the training dataset, so a total of 60000 examples.
model_v2.fit(norm_train_dataset, epochs=1, steps_per_epoch=math.ceil(num_train_examples/BATCH_SIZE))

Train for 1875 steps
1875/1875 [==============================] - 12s 7ms/step - loss: 1.6788 - accuracy: 0.7919


In [0]:
# Evaluate accuracy
test_loss, test_accuracy = model_v2.evaluate(norm_test_dataset, steps=math.ceil(num_test_examples/BATCH_SIZE))
print('Accuracy on test dataset:', test_accuracy)

Restarting runtime resolved the issue with flatten_2_input dimensions. It seems that flatten function change is inplace in the dataset.

After changing epochs from 5 to 1, the accuracy decreased from 0.8687 to 0.7019 for train dataset and from 0.8355 to 0.8280 for test dataset.

## Change Number of neurons in the Dense layer from 128 to 10

In [13]:
model_v3 = build_model(10)

# Train the model
model_v3.fit(norm_train_dataset, epochs=5, steps_per_epoch=math.ceil(num_train_examples/BATCH_SIZE))

Train for 1875 steps
Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 1.7954 - accuracy: 0.6910
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 1.7037 - accuracy: 0.7669
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 1.6796 - accuracy: 0.7897
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 1.6268 - accuracy: 0.8420
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 1.6203 - accuracy: 0.8478


In [14]:
# Evaluate accuracy
test_loss, test_accuracy = model_v3.evaluate(norm_test_dataset, steps=math.ceil(num_test_examples/BATCH_SIZE))
print('Accuracy on test dataset:', test_accuracy)

313/313 [==============================] - 1s 2ms/step - loss: 1.6328 - accuracy: 0.8326
Accuracy on test dataset: 0.8326


After changing the hidden layer neuron number from 128 to 10, the accuracy changed from 0.8687 to 0.8478 for train dataset and from 0.8355 to 0.8326 for test dataset.

## Change Number of neurons in the Dense layer from 128 to 512

In [15]:
model_v4 = build_model(512)

# Train the model
model_v4.fit(norm_train_dataset, epochs=5, steps_per_epoch=math.ceil(num_train_examples/BATCH_SIZE))

Train for 1875 steps
Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 1.7158 - accuracy: 0.7481
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 1.6733 - accuracy: 0.7881
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 1.6190 - accuracy: 0.8422
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 1.6070 - accuracy: 0.8546
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 1.6022 - accuracy: 0.8588


In [16]:
# Evaluate accuracy
model_v4.evaluate(norm_test_dataset, steps=math.ceil(num_test_examples/BATCH_SIZE))

313/313 [==============================] - 1s 2ms/step - loss: 1.6277 - accuracy: 0.8323


[1.627653985739516, 0.8323]

## Add 2nd hidden dense layer with 128 neurons

In [21]:
model_v5 = build_model(hidden2=True)

# Train the model
model_v5.fit(norm_train_dataset, epochs=5, steps_per_epoch=math.ceil(num_train_examples/BATCH_SIZE))

Train for 1875 steps
Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 1.6889 - accuracy: 0.7769
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 1.6298 - accuracy: 0.8316
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 1.6173 - accuracy: 0.8437
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 1.6059 - accuracy: 0.8553
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 1.6029 - accuracy: 0.8578


In [22]:
# Evaluate accuracy
model_v5.evaluate(norm_test_dataset, steps=math.ceil(num_test_examples/BATCH_SIZE))

313/313 [==============================] - 1s 2ms/step - loss: 1.6114 - accuracy: 0.8501


[1.6113796447412656, 0.8501]

## Add 2nd hidden dense layer with 50 neurons

In [19]:
model_v6 = build_model(hidden2_num=50, hidden2=True)

# Train the model
model_v6.fit(norm_train_dataset, epochs=5, steps_per_epoch=math.ceil(num_train_examples/BATCH_SIZE))

Train for 1875 steps
Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 1.6694 - accuracy: 0.7981
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 1.6248 - accuracy: 0.8368
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 1.6134 - accuracy: 0.8477
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 1.6022 - accuracy: 0.8589
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 1.6013 - accuracy: 0.8594


In [20]:
# Evaluate accuracy
model_v6.evaluate(norm_test_dataset, steps=math.ceil(num_test_examples/BATCH_SIZE))

313/313 [==============================] - 1s 2ms/step - loss: 1.6143 - accuracy: 0.8470


[1.6143450893152256, 0.847]

## Set 1st hidden layer neuron to 512 and 2nd hidden layer neuron to 128

In [28]:
model_v7 = build_model(hidden1_num=512, hidden2_num=128, hidden2=True)

# Train the model
model_v7.fit(norm_train_dataset, epochs=5, steps_per_epoch=math.ceil(num_train_examples/BATCH_SIZE))

Train for 1875 steps
Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 1.7140 - accuracy: 0.7483
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 1.6869 - accuracy: 0.7732
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 1.6623 - accuracy: 0.7980
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 1.6274 - accuracy: 0.8333
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 1.6188 - accuracy: 0.8418


In [29]:
# Evaluate accuracy
model_v7.evaluate(norm_test_dataset, steps=math.ceil(num_test_examples/BATCH_SIZE))

313/313 [==============================] - 1s 2ms/step - loss: 1.6333 - accuracy: 0.8282


[1.6333180128956755, 0.8282]

## Use data without normalization

In [32]:
model_v8 = build_model()

# Train the model
# Use train_dataset instead of norm_train_dataset
model_v8.fit(train_dataset, epochs=5, steps_per_epoch=math.ceil(num_train_examples/BATCH_SIZE))

Train for 1875 steps
Epoch 1/5
1875/1875 [==============================] - 5s 3ms/step - loss: 2.3594 - accuracy: 0.1017
Epoch 2/5
1875/1875 [==============================] - 5s 2ms/step - loss: 2.3626 - accuracy: 0.0985
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 2.3615 - accuracy: 0.0997
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 2.3608 - accuracy: 0.1004
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 2.3604 - accuracy: 0.1007


In [33]:
# Evaluate accuracy
# Use test_dataset instead of norm_test_dataset
model_v8.evaluate(test_dataset, steps=math.ceil(num_test_examples/BATCH_SIZE))

313/313 [==============================] - 2s 7ms/step - loss: 2.3611 - accuracy: 0.1000


[2.361110233270322, 0.1]

## Exercise Summary
The modifications on hidden layers and epochs did not impact much on the accuracy, but the data normalization had a huge impact on the accuracy.

model version |	modification | accuracy on train |	accuracy on test
--- | --- | --- | ---
model |	original | 0.8683 | 0.8355
model_v2 | Set training epochs set to 1	| 0.7919 | 0.828
model_v3 |	Change Number of neurons in the Dense layer from 128 to 10 | 0.8478 | 0.8326
model_v4 |	Change Number of neurons in the Dense layer from 128 to 512 |0.8588 | 0.8323
model_v5 |	Add 2nd hidden dense layer with 128 neurons | 0.8578 | 0.8501
model_v6 |	Add 2nd hidden dense layer with 50 neurons | 0.8594 | 0.847
model_v7 |	Set 1st hidden layer neuron to 512 and 2nd hidden layer neuron to 128 |	0.8418 | 0.8282
model_v8 |	Use data without normalization | 0.1007 | 0.1